In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "Simple App")

sc

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Simple App, master=local) created by __init__ at <ipython-input-2-1269fd746409>:2 

In [1]:
# Original post and context: 
# http://www.supergloo.com/fieldnotes/apache-spark-quick-start-with-python-new-york-city-uber-trips


In [1]:
ut = sc.textFile("Uber-Jan-Feb-FOIL.csv")
ut.count()

NameError: name 'sc' is not defined

In [3]:
ut.first()

u'dispatching_base_number,date,active_vehicles,trips'

In [4]:
rows = ut.map(lambda line: line.split(","))
rows.map(lambda row: row[0]).distinct().count()

7

In [5]:
rows.map(lambda row: row[0]).distinct().collect()

[u'B02617',
 u'B02682',
 u'B02598',
 u'B02765',
 u'B02512',
 u'dispatching_base_number',
 u'B02764']

In [6]:
rows.filter(lambda row: "B02617" in row).count()

59

In [7]:
base02617 = rows.filter(lambda row: "B02617" in row)
base02617.filter(lambda row: int(row[3]) > 15000).count()

6

In [8]:
base02617.filter(lambda row: int(row[3]) > 15000).map(lambda day: day[1]).distinct().count()

6

In [9]:
filteredRows = sc.textFile("Uber-Jan-Feb-FOIL.csv").filter(lambda line: "base" not in line).map(lambda line:line.split(","))
filteredRows.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).collect()

[(u'B02617', 725025),
 (u'B02682', 662509),
 (u'B02598', 540791),
 (u'B02765', 193670),
 (u'B02512', 93786),
 (u'B02764', 1914449)]

In [10]:
filteredRows.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).takeOrdered(10, key=lambda x: -x[1])

[(u'B02764', 1914449),
 (u'B02617', 725025),
 (u'B02682', 662509),
 (u'B02598', 540791),
 (u'B02765', 193670),
 (u'B02512', 93786)]